<a href="https://colab.research.google.com/github/yougoal/leejunghoon/blob/master/DQN2015_other.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import gym
from gym import wrappers
import numpy as np
import random as ran

env = gym.make('CartPole-v0')

# 꺼내서 사용할 리플레이 갯수
REPLAY = 3
# 리플레이를 저장할 리스트
REPLAY_MEMORY = []
# 미니배치
MINIBATCH = 5

INPUT = env.observation_space.shape[0]
OUTPUT = env.action_space.n

# 하이퍼파라미터
LEARNING_LATE = 0.001
DISCOUNT = 0.99
model_path = "save/model.ckpt"


# 두개의 네트워크 구성

x=tf.placeholder(dtype=tf.float32, shape=(None, 4))
Y=tf.placeholder(dtype=tf.float32, shape=(None, 2))


# Main 네트워크
W1 = tf.get_variable('W1',shape=[4,2],initializer=tf.contrib.layers.xavier_initializer())
Q_pre = tf.matmul(x,W1)

# Target 네트워크
W1_r = tf.get_variable('W1_r',shape=[4,2],initializer=tf.contrib.layers.xavier_initializer())
Q_pre_r = tf.matmul(x,W1_r)

# 총 Reward를 저장해놓을 리스트
rlist=[0]
recent_rlist=[0]

episode = 0

# Loss function 정의
cost = tf.reduce_sum(tf.square(Y-Q_pre))
optimizer = tf.train.AdamOptimizer(LEARNING_LATE, epsilon=0.01)
train = optimizer.minimize(cost)


saver = tf.train.Saver()

# 세션 정의
with tf.Session() as sess:
    # 변수 초기화
    sess.run(tf.global_variables_initializer())
    # Target 네트워크에 main 네트워크 값을 카피해줌
    sess.run(W1_r.assign(W1))
    print("초기시작시 Wm과 Wt확인")
    print(sess.run(W1_r.assign(W1)))
    print(sess.run(W1))

    # 에피소드 시작
    while np.mean(recent_rlist) <10 :
        episode += 1

        # state 초기화
        s = env.reset()
        if len(recent_rlist) > 5:
            del recent_rlist[0]
        print("recent_rlist")
        print(recent_rlist)
        # e-greedy
        e = 1. / ((episode/25)+1)

        rall = 0
        d = False
        count = 0

        # 에피소드가 끝나기 전까지 반복
        while not d  :

            #env.render()
            count += 1

            # state 값의 전처리
            s_t = np.reshape(s,[1,INPUT])
            # 현재 상태의 Q값을 에측
            Q = sess.run(Q_pre, feed_dict={x:s_t})
            # e-greedy 정책으로 랜덤하게 action 결정
            if e > np.random.rand(1):
                a = env.action_space.sample()
            else:
                a = np.argmax(Q)

            # 결정된 action으로 Environment에 입력
            s1, r, d, _ = env.step(a)

            # Environment에서 반환한 Next_state, action, reward, done 값들을
            # Replay_memory에 저장
            REPLAY_MEMORY.append([s_t,a,r,s1,d,count])

            # 저장된 값들이 50000개 이상 넘어가면 맨 앞 Replay부터 삭제
            if len(REPLAY_MEMORY) > 5:
                del REPLAY_MEMORY[0]
            print('메모리 버퍼에 저장된 값 출력')
           # print(REPLAY_MEMORY[0:1])
            #print(REPLAY_MEMORY[1:2])
            #print(REPLAY_MEMORY[2:3])
            #print(REPLAY_MEMORY[3:4])
            #print(REPLAY_MEMORY[4:5])
            #print(REPLAY_MEMORY[5:6])
            # 총 reward 합
            rall += r
            # state를 Next_state로 바꿈
            s = s1


        # 일정간격의 episode마다 학습을 수행한다.
        if episode % 5 == 2 :

            # 50번의 미니배치로 학습
                # 저장된 리플레이 중에 학습에 사용할 랜덤한 리플레이 샘플들을 가져옴
            for sample in ran.sample(REPLAY_MEMORY, 2):
                s_t_r, a_r, r_r, s1_r, d_r ,count_r= sample
                print("메모리버퍼에서 꺼내온 샘플 Target Str At Rt St+1 done count_R")
                print(sample)
                # 꺼내온 리플레이의 state의 Q값을 예측
                Q = sess.run(Q_pre, feed_dict={x: s_t_r})
                print("메모리버퍼에서 꺼내온 St로 Q(St,a)를 예측 Qpre=st*Wm")
                print(sess.run(W1))
                if d_r:
                    # 꺼내온 리플레이의 상태가 끝난 상황이라면 Negative Reward를 부여
                        Q[0, a_r] = -100
                else:
                    # 끝나지 않았다면 Q값을 업데이트
                    # 꺼내온 st+1로 표적의 값을 만들어낸다
                    s1_t_r= np.reshape(s1_r,[1,4])
                    Q1 = sess.run(Q_pre_r, feed_dict={x: s1_t_r})
                    print("y(st.a)에 사용된 Wt")
                    print(sess.run(W1))
                    Q[0, a_r]=np.max(Q1)
                    # 표적값
                    print("표적 Q(st+1,at)==>y(st,a)=st+1*Wt, Q1 및 Q")
                    print(Q1,Q)
                    
                # 업데이트 된 Q값으로 main네트워크를 학습
                _, loss = sess.run([train, cost], feed_dict={x: s_t_r, Y: Q })
                print("훈련에참여한 값 St{}".format(s_t_r))
                print("훈련에참여한 값 Target{}".format(Q))
                print(" REPLY 훈련W값")
                print(sess.run([W1]))
                print("최종 loss{}".format(loss))
                
            # 10번 마다 target 네트워크에 main 네트워크 값을 copy
            sess.run(W1_r.assign(W1))
            print("Wm==>Wt로 복사")
            print(sess.run(W1_r.assign(W1)))
        # 총 reward의 합을 list에 저장
        recent_rlist.append(rall)
        rlist.append(rall)
        print("Episode:{} steps:{} reward:{} average reward:{} recent reward:{}".format(episode, count, rall,
                                                                                        np.mean(rlist),
                                                                                        np.mean(recent_rlist)))

    save_path = saver.save(sess, model_path)
    print("Model saved in file: ",save_path)


    rlist=[]
    recent_rlist=[]


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, model_path)

    print("Model restored form file: ", save_path)
    for episode in range(10):
        # state 초기화
        s = env.reset()

        rall = 0
        d = False
        count = 0
        # 에피소드가 끝나기 전까지 반복
        while not d :
           # env.render()
            count += 1
            # state 값의 전처리
            s_t = np.reshape(s, [1, INPUT])

            # 현재 상태의 Q값을 에측
            Q = sess.run(Q_pre, feed_dict={x: s_t})
            a = np.argmax(Q)

            # 결정된 action으로 Environment에 입력
            s, r, d, _ = env.step(a)

            # 총 reward 합
            rall += r


        rlist.append(rall)

        print("Episode : {} steps : {} r={}. averge reward : {}".format(episode, count, rall,
                                                                        np.mean(rlist)))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


초기시작시 Wm과 Wt확인
[[ 0.89659953  0.08986282]
 [-0.9402487  -0.43435168]
 [ 0.5121553  -0.33361053]
 [-0.9986341   0.3573928 ]]
[[ 0.89659953  0.08986282]
 [-0.9402487  -0.43435168]
 [ 0.5121553  -0.33361053]
 [-0.9986341   0.3573928 ]]
recent_rlist
[0]
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
Episode:1 steps:13 reward:13.0 average reward:6.5 recent reward:6.5
recent_rlist
[0, 13.0]
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값 출력
메모리 버퍼에 저장된 값